In [1]:
# import gymnasium as gym
import gym
import numpy as np
import pygame
from gym import spaces
from vaccum_env import VacEnv

room_maze=np.array(
    [
        ['S','0','1','1','0','0','0','0','0','0','0'],
        ['0','0','0','0','0','1','0','1','1','1','0'],
        ['0','0','1','1','0','1','0','1','1','1','0'],
        ['1','0','1','1','0','1','0','G','0','0','0'],
        ['1','0','0','0','0','1','0','0','0','0','0']
    ]
)

gym.envs.register(id='VacEnv',entry_point='vaccum_env:VacEnv',kwargs={'grid':room_maze})

env=gym.make('VacEnv')

TypeError: VacEnv() takes no arguments

In [3]:
class VacEnv(gym.Env):
  def _init_(self):
    super(VacEnv,self).__init__()
    self.room=room_maze
    self.start=np.where(self.room=='S')
    self.goal=np.where(self.room=='G')
    self.action_space=spaces.Discrete(4)
    self.rows,self.cols=self.room.shape
    self.observation_space=spaces.Tuple(spaces.Discrete(self.rows),spaces.Discrete(self.cols))

    pygame.init()
    self.cell_size = 170
    self.screen = pygame.display.set_mode((self.cols * self.cell_size, self.rows * self.cell_size))

    def reset(self):
        self.current=self.start
        return self.current

  def step(self,action):
    new_pos=np.array(self.current)
    if action==0:
      new_pos[0]-=1
    elif action==1:
       new_pos[0]+=1
    elif action==2:
       new_pos[1]-=1
    elif action==3:
       new_pos[1]+=1
    if self._is_valid_position(new_pos):
      self.current=new_pos
    if np.array_equal(self.current,self.goal):
      reward=1
      done=True
    else:
      reward=0
      done=False
    return self.current,reward,done,{}
    def _is_valid_position(self,pos):
      row,col=pos
      if row<0 or col<0 or row>= self.rows or col >=self.cols:
        return False
      if self.room[row,col]=='1':
       return False
      return True
    def render(self):
      self.screen.fill((255, 255, 255))

      for row in range(self.rows):
          for col in range(self.cols):
              cell_left = col * self.cell_size
              cell_top = row * self.cell_size

              try:
                  print(np.array(self.current_pos)==np.array([row,col]).reshape(-1,1))
              except Exception as e:
                  print('Initial state')

              if self.maze[row, col] == '1':
                  pygame.draw.rect(self.screen, (0, 0, 0), (cell_left, cell_top, self.cell_size, self.cell_size))
              elif self.maze[row, col] == 'S':
                  pygame.draw.rect(self.screen, (0, 255, 0), (cell_left, cell_top, self.cell_size, self.cell_size))
              elif self.maze[row, col] == 'G':
                  pygame.draw.rect(self.screen, (255, 0, 0), (cell_left, cell_top, self.cell_size, self.cell_size))

              if np.array_equal(np.array(self.current_pos), np.array([row, col]).reshape(-1,1)):
                  pygame.draw.rect(self.screen, (0, 0, 255), (cell_left, cell_top, self.cell_size, self.cell_size))

      pygame.display.update()

In [ ]:
# Define the environment
environment = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])

# Define the Q-table
q_table = np.zeros((3, 3))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to display the environment and vacuum cleaner position
def display_environment(state):
    symbols = {0: ' ', 1: 'D'}
    for i in range(3):
        row = ''
        for j in range(3):
            if i == state and environment[i][j] == 1:
                row += 'V'
            else:
                row += symbols[environment[i][j]]
        print(row)
    print()

# Create the gym environment
env = gym.make('FrozenLake-v0')

# Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, _ = env.step(action)

        q_table[state][action] = (1 - learning_rate) * q_table[state][action] + learning_rate * (reward + discount_factor * np.max(q_table[next_state]))

        state = next_state

        # Print the current state of the environment
        print("Episode:", episode+1)
        display_environment(state)
        print("Action:", action)
        print("Next State:", next_state)
        print("Reward:", reward)
        print("Q-table:")
        print(q_table)
        print("--------------------")

# Print the learned Q-table
print("Learned Q-table:")
print(q_table)


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment FrozenLake-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


DeprecatedEnv: ignored